In [5]:
import re

def replace_first_instance(string, forward=True):
    """Replace first instance of a spelled-out digit."""
    digits = {'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'}
    for i in range(len(string)):
        # Return unchanged string if numeric digit appears first.
        if string[i].isdigit():
            return string
        for digit in digits:
            # Forward search and replace
            if forward and digit in string[:i+1]:
                return string.replace(digit, digits[digit], 1)
            # Backward search and replace
            if not forward and digit[::-1] in string[:i+1]:
                return string.replace(digit[::-1], digits[digit], 1)
    # Return unchanged string if no spelled-out digit appears.
    return string
    
def day_1(input_file):
    """Solve day 1 puzzle of Advent of Code 2023."""
    sums = [0, 0]
    with open(input_file, 'r') as file:
        for line in file:
            numbers = re.findall(r'\d', line)
            sums[0] += int(numbers[0] + numbers[-1])
            # Replace first instance of a spelled-out digit
            line = replace_first_instance(line)
            # Replace last instance of a spelled-out digit
            line = replace_first_instance(line[::-1], forward=False)[::-1]
            numbers = re.findall(r'\d', line)
            sums[1] += int(numbers[0] + numbers[-1])
    return sums

day_1('Day1.txt')  

[55477, 54431]

In [6]:
def day_2(input_file):
    """Solve day 2 puzzle of Advent of Code 2023."""
    part_1 = {'red': 12, 'green': 13, 'blue': 14}
    ID_sum = 0 
    powers = 0
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            game = line.strip('\n').split(':', 1)[1].split(';')
            game_possible = True
            part_2 = {'red': 0, 'green': 0, 'blue': 0}
            for cubes in game:
                for cube in cubes.split(','):
                    number, color = cube.split(' ')[1:]
                    number = int(number)
                    if number > part_1[color]:
                        game_possible = False
                    part_2[color] = max(number, part_2[color])                       
            if game_possible:
                ID_sum += i+1
            powers += part_2['red'] * part_2['green'] * part_2['blue']
    return (ID_sum, powers)

day_2('Day2.txt')

(2528, 67363)

In [7]:
def day_3(input_file):
    """Solve day 3 puzzle of Advent of Code 2023."""
    nbors = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    parts = ()
    neighbours = ()
    symbols = {}
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            part = ''
            neighbours_part = set()
            for j, char in enumerate(line):
                if char.isdigit():
                    part += char
                    neighbours_part.update([(i+nbor[0],j+nbor[1]) for nbor in nbors]) 
                else:
                    if part != '':
                        parts += (part,)
                        neighbours += (neighbours_part,)
                        part = ''
                        neighbours_part = set()                        
                    if char != '.' and char != '\n':
                        symbols[(i, j)] = char
                        
    sum = 0
    for i, part in enumerate(parts):
        for neighbour in neighbours[i]:
            if neighbour in symbols:
                sum += int(part)
                break
            
    gear_ratio = 0
    for coord in [key for key, value in symbols.items() if value == '*']:
        parts_gear = ()
        for i, neighbour in enumerate(neighbours):
            if coord in neighbour: 
                parts_gear += (int(parts[i]),)
        if len(parts_gear) == 2:
            gear_ratio += parts_gear[0] * parts_gear[1]
        
    return (sum, gear_ratio)                
    
day_3('Day3.txt')
    

(559667, 86841457)

In [8]:
def day_4(input_file):
    """Solve day 4 puzzle of Advent of Code 2023.""" 
    sum = 0
    num_cards = 0
    stash = []      
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            winning_nums, nums = line.strip('\n').split(': ')[1].split(' | ')
            winning_nums = [int(num) for num in winning_nums.split()]
            nums = [int(num) for num in nums.split()]
            matching_nums = len([num for num in nums if num in winning_nums])
            sum += int(2**matching_nums-1)
            num_card = 1
            if len(stash) > 0:
                num_card += stash[0]
                stash.pop(0)
            num_cards += num_card
            for i in range(matching_nums):
                try:
                    stash[i] += num_card
                except IndexError:
                    stash.append(num_card)
    return((sum, num_cards))

day_4('Day4.txt')
    

(51131, 19499881)

In [9]:
def day_5(input_file):
    """Solve day 5 puzzle of Advent of Code 2023."""  
    with open(input_file, 'r') as file:
        for line in file:
            if line.startswith('seeds'):
                numbers_1 = [int(number) for number in line.strip('seeds: ').strip('\n').split()]
                numbers_2 = [(a, b) for a, b in zip(numbers_1[::2], numbers_1[1::2])]
            elif line[0].isalpha():
                try:
                    numbers_1 = new_numbers_1.copy()
                    numbers_2 = new_numbers_2.copy()
                except NameError:
                    pass
                new_numbers_1 = numbers_1.copy()
                new_numbers_2 = numbers_2.copy()
            elif line[0].isdigit():
                dest, source, length = [int(number) for number in line.strip('\n').split()]
                for i, number in enumerate(numbers_1):
                    if number >= source and number < source + length:
                        new_numbers_1[i] = dest - source + number
                for i, (number, range) in enumerate(numbers_2):
                    # If overlap but number < source, cut off part of
                    # (number, range) < source
                    if number < source and number + range > source:
                        numbers_2.append((number, source - number))
                        new_numbers_2.append((number, source - number))  
                        range -= source - number 
                        number = source
                    # If overlap but number + range > source + length, cut off part of
                    # (number, range) > source + length
                    if number < source + length and number + range > source + length:
                        numbers_2.append((source + length, number + range - source - length))
                        new_numbers_2.append((source + length, number + range - source - length))  
                        range -= number + range - source - length
                    if number >= source and number < source + length:
                        numbers_2[i] = (number, range)
                        new_numbers_2[i] = (dest - source + number, range)
                    
    return (min(new_numbers_1), min([a for (a, b) in new_numbers_2]))  
                 
day_5('Day5.txt')

(177942185, 69841803)

In [10]:
def day_6(input_file):
    """Solve day 6 puzzle of Advent of Code 2023."""
    product= 1
    with open(input_file, 'r') as file:
        for line in file:
            if line.startswith('Time:'):
                times = [int(time) for time in line.strip('\n').strip('Time:').split()]
                times.append(int(line.strip('\n').strip('Time:').replace(' ', '')))
            elif line.startswith('Distance:'):
                distances = [int(time) for time in line.strip('\n').strip('Distance:').split()]
                distances.append(int(line.strip('\n').strip('Distance:').replace(' ', '')))
    # time = time_move + time_charge
    # distance = time_move * speed = time_move * time_charge
    for time, distance in zip(times, distances):
        number = 0
        for time_charge in range(time):
            dist = time_charge * (time - time_charge)
            if dist > distance:
                number += 1
        if time == times[-1]:
            return (product, number)
        product *= number
        
day_6('Day6.txt')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/ff/_83w57j50dbbqcv7hghzlg880000gn/T/ipykernel_64441/2763250825.py", line 24, in <module>
    day_6('Day6.txt')
  File "/var/folders/ff/_83w57j50dbbqcv7hghzlg880000gn/T/ipykernel_64441/2763250825.py", line -1, in day_6
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/opt/homebrew/Caskroom/miniconda/base/

In [ ]:
def sort_hands_1(hands):
    """Sort hands first by type and then by card value."""
    hands_sort = []
    for hand in hands:
        # Replace A, K, Q, J, T with E, D, C, B, A to enable sorting
        hand_sort = hand.replace('A', 'E').replace('K', 'D').replace('Q', 'C').replace('J', 'B').replace('T', 'A')
        # High card
        if len(set(hand)) == 5:
            hands_sort.append('1' + hand_sort)
        # One pair
        elif len(set(hand)) == 4:
            hands_sort.append('2' + hand_sort)
        # Two pair
        elif len(set(hand)) == 3 and max([hand.count(char) for char in hand]) == 2:
            hands_sort.append('3' + hand_sort)
        # Three of a kind
        elif len(set(hand)) == 3 and max([hand.count(char) for char in hand]) == 3:
            hands_sort.append('4' + hand_sort)
        # Full house
        elif len(set(hand)) == 2 and max([hand.count(char) for char in hand]) == 3:
            hands_sort.append('5' + hand_sort)
        # Four of a kind
        elif len(set(hand)) == 2 and max([hand.count(char) for char in hand]) == 4:
            hands_sort.append('6' + hand_sort)
        # Five of a kind
        elif len(set(hand)) == 1:
            hands_sort.append('7' + hand_sort)
    hands_sort.sort()
    # Revert replacing
    hands = [hand[1:].replace('A', 'T').replace('E', 'A').replace('D', 'K').replace('C', 'Q').replace('B', 'J') for hand in hands_sort]
    return hands

def sort_hands_2(hands):
    """Sort hands first by type and then by card value."""
    hands_sort = []
    for hand in hands:
        # Replace A, K, Q, J, T with E, D, C, B, A to enable sorting
        hand_sort = hand.replace('A', 'E').replace('K', 'D').replace('Q', 'C').replace('J', '1').replace('T', 'A')
        # High card
        if len(set(hand)) == 5:
            # If joker, then one pair
            rank = 1 + hand_sort.count('1')
        # One pair
        elif len(set(hand)) == 4:
            # If one or two jokers, then three of a kind
            if hand_sort.count('1') > 0:
                rank = 4
            else:
                rank = 2
        # Two pair
        elif len(set(hand)) == 3 and max([hand.count(char) for char in hand]) == 2:
            # If one joker, then full house; if two jokers, then four of a kind
            if hand_sort.count('1') > 0:
                rank = 4 + hand_sort.count('1')
            else:
                rank = 3
        # Three of a kind
        elif len(set(hand)) == 3 and max([hand.count(char) for char in hand]) == 3:
            # If one or three jokers, then four of a kind
            if hand_sort.count('1') > 0:
                rank = 6
            else:
                rank = 4
        # Full house
        elif len(set(hand)) == 2 and max([hand.count(char) for char in hand]) == 3:
            # If two or three jokers, then five of a kind
            if hand_sort.count('1') > 0:
                rank = 7
            else:
                rank = 5
        # Four of a kind
        elif len(set(hand)) == 2 and max([hand.count(char) for char in hand]) == 4:
            # If one or four jokers, then five of a kind
            if hand_sort.count('1') > 0:
                rank = 7
            else:
                rank = 6
        # Five of a kind
        elif len(set(hand)) == 1:
            rank = 7
        hands_sort.append(str(rank) + hand_sort)
    hands_sort.sort()
    # Revert replacing
    hands = [hand[1:].replace('A', 'T').replace('E', 'A').replace('D', 'K').replace('C', 'Q').replace('1', 'J') for hand in hands_sort]
    return hands
        
def day_7(input_file):
    """Solve day 7 puzzle of Advent of Code 2023."""
    with open(input_file, 'r') as file:
        hands = {}
        for line in file:
            hand, winning = line.strip('\n').split()
            hands[hand] = int(winning)
    hands_sorted = sort_hands_1(hands.keys())
    winnings_1 = sum([(i+1) * hands[hand] for i, hand in enumerate(hands_sorted)])
    hands_sorted = sort_hands_2(hands.keys())
    winnings_2 = sum([(i+1) * hands[hand] for i, hand in enumerate(hands_sorted)])
    return (winnings_1, winnings_2)         

day_7('Day7.txt')

(250951660, 251481660)

In [ ]:
import itertools
import numpy

def day_8(input_file):
    """Solve day 8 puzzle of Advent of Code 2023."""
    neighbours = {}
    pos_2 = []
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            if i == 0:
                instructs = [int(number) for number in line.strip('\n').replace('L', '0').replace('R', '1')]
            elif i >= 2:
                source, dests = line.strip('\n').split(' = ')
                dests = tuple(dest for dest in dests.strip('(').strip(')').split(', '))
                neighbours[source] = dests
                if source[2] == 'A':
                    pos_2.append(source)
                    
    pos_1 = 'AAA'
    steps_2 = []
    for i, instruct in enumerate(itertools.cycle(instructs)):
        pos_1 = neighbours[pos_1][instruct]
        if pos_1 == 'ZZZ':
            steps_1 = i + 1
        delete = ()
        for j, pos in enumerate(pos_2):
            new_pos = neighbours[pos][instruct]
            pos_2[j] = new_pos
            if new_pos[2] == 'Z':
                steps_2.append(i + 1)
                delete += (j,)
        for j in delete:
            pos_2.pop(j)
        if all([pos == 0 for pos in pos_2]):
            break
    steps_2 = numpy.lcm.reduce(steps_2)
    return (steps_1, steps_2)
          
day_8('Day8.txt')    

(15871, 11283670395017)

In [29]:
def day_9(input_file):
    """Solve day 9 puzzle of Advent of Code 2023."""
    sum_1 = 0
    sum_2 = 0
    with open(input_file, 'r') as file:
        for line in file:
            diff = tuple(int(number) for number in line.strip('\n').split())
            last = (diff[-1],)
            first = (diff[0],)
            while len(set(diff)) > 1:
                diff = tuple(diff[i+1]-diff[i] for i in range(len(diff)-1))
                last += (diff[-1],)
                first += (diff[0],)
            sum_1 += sum(last)
            extrapolate = 0
            for number in first[::-1]:
                extrapolate = number - extrapolate
            sum_2 += extrapolate
            
    return (sum_1, sum_2)
          
day_9('Day9.txt')  

(2038472161, 1091)

In [81]:
def day_10(input_file):
    """Solve day 10 puzzle of Advent of Code 2023."""
    # Read in map
    pipes = {}
    with open(input_file, 'r') as file:
        for y, line in enumerate(file):
            for x, char in enumerate(line.strip('\n')):
                if char == '|':
                    pipes[(x, y)] = ((x, y - 1), (x, y + 1))
                elif char == '-':
                    pipes[(x, y)] = ((x - 1, y), (x + 1, y))
                elif char == 'L':
                    pipes[(x, y)] = ((x, y - 1), (x + 1, y))
                elif char == 'J':
                    pipes[(x, y)] = ((x, y - 1), (x - 1, y))
                elif char == '7':
                    pipes[(x, y)] = ((x, y + 1), (x - 1, y))
                elif char == 'F':
                    pipes[(x, y)] = ((x, y + 1), (x + 1, y))
                elif char == 'S':
                    start = (x, y)
    
    # Find pipes connecting to start
    pipes[start] = ()
    for neighbour in ((start[0], start[1] - 1), (start[0], start[1] + 1),
        (start[0] - 1, start[1]), (start[0] + 1, start[1])):
        if neighbour in pipes and start in pipes[neighbour]:
            pipes[start] += (neighbour,)
    
    # Continue to unvisited neighbour until back at start
    pos = pipes[start][0]
    visited = (start,)
    step = 1
    steps = 0
    while steps == 0:
        step += 1
        for neighbour in pipes[pos]:
            if step > 2 and neighbour == start:
                steps = step // 2
            if neighbour not in visited:
                visited += (pos,)                
                pos = neighbour
    
    # Pick's theorem and shoelace formula
    number = len(visited)
    sum = 0
    for i in range(number):
        sum += visited[i][0] * visited[(i + 1) % number][1] - \
            visited[(i + 1) % number][0] * visited[i][1]
            
    return (steps, abs(sum) // 2 - steps + 1)

day_10('Day10.txt')  

(7086, 317)

In [21]:
def day_11(input_file, expansion_factor):
    """Solve day 11 puzzle of Advent of Code 2023."""
    # Read in galaxies and empty rows and columns
    galaxies = []
    expand_x = set()
    expand_y = set()
    with open(input_file, 'r') as file:
        for y, line in enumerate(file):
            expand_y.add(y)
            for x, char in enumerate(line.strip('\n')):
                if y == 0:
                    expand_x.add(x)
                if char == '#':
                    galaxies.append((x, y))
                    expand_x.discard(x)
                    expand_y.discard(y)

    # Expand universe
    for i, (x, y) in enumerate(galaxies):
        x += len([x_pand for x_pand in expand_x if x_pand < x]) * (expansion_factor - 1)
        y += len([y_pand for y_pand in expand_y if y_pand < y]) * (expansion_factor - 1)
        galaxies[i] = (x, y)
    
    # Length of shortest path is simply Euclidean distance
    sum = 0
    for (galaxy_1, galaxy_2) in itertools.combinations(galaxies, 2):
        sum += abs(galaxy_1[0] - galaxy_2[0]) + \
            abs(galaxy_1[1] - galaxy_2[1])
        
    return sum   
    
print(day_11('Day11.txt', 2))
print(day_11('Day11.txt', 1000000))

9627977
644248339497


In [12]:
from functools import cache

@cache
def find_arrangements(record_1, record_2, number_damaged=0):
    if record_1 == '':
        # Possible arrangement?
        if len(record_2) == 0 and number_damaged == 0:
            return 1
        else:
            return 0
    number_arrangements = 0
    if record_1[0] == '#' or record_1[0] == '?':
        # Extend group of broken springs
        number_arrangements += find_arrangements(record_1[1:], record_2, number_damaged + 1)
    if record_1[0] == '.' or record_1[0] == '?':
        # End group of broken springs
        # Possible arrangement?
        if number_damaged > 0:
            if len(record_2) > 0 and record_2[0] == number_damaged:
                number_arrangements += find_arrangements(record_1[1:], record_2[1:])  
        # Next spring          
        else:
            number_arrangements += find_arrangements(record_1[1:], record_2)
    return number_arrangements

def day_12_1(input_file):
    """Solve part 1 of day 12 puzzle of Advent of Code 2023."""
    total = 0
    with open(input_file, 'r') as file:  
        for line in file:
            record_1, record_2 = line.strip('\n').split()
            record_2 = tuple(eval(record_2))
            number_damaged = sum(record_2) - record_1.count('#')
            unknown = tuple(i for i, char in enumerate(record_1) if char == '?')
            for damaged in tuple(itertools.combinations(unknown, number_damaged)):
                arrangement = record_1
                for i in damaged:
                    arrangement = arrangement[:i] + '#' + arrangement[i + 1:] 
                arrangement = arrangement.replace('?', '.')
                if tuple(len(string) for string in arrangement.split('.') if string != '') == record_2:
                    total += 1
    return total

def day_12_2(input_file, copies):
    """Solve part 2 of day 12 puzzle of Advent of Code 2023."""
    total = 0
    with open(input_file, 'r') as file:  
        for line in file:
            record_1, record_2 = line.strip('\n').split()
            # Add . to simply finding arrangements
            record_1 = '?'.join([record_1] * copies) + '.'
            record_2 = tuple(eval(record_2)) * copies
            total += find_arrangements(record_1, record_2)  
    return total
                
print(day_12_1('Day12.txt'))
print(day_12_2('Day12.txt', 5))

8180
620189727003627
